In [1]:
import json
import logging
import pickle
from pathlib import Path

import chromadb
import numpy as np
from tqdm import tqdm

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

## Initialization

In [3]:
# directory, where Chroma will store collections
CHROMADB_STORAGE_FOLDER = Path("/home/deniskirbaba/Documents/legal-ai/storage")

# directory with data, used to init legal practice collection (i.e. documents, embeddings, metadata)
LEGAL_PRACTICES_DATA_FOLDER = Path("/home/deniskirbaba/Documents/legal-ai/practice_db/data")


def squeeze_meta(meta: dict) -> dict:
    """
    Convert metadatas lists to strings
    """
    meta["codex"] = "".join(meta["codex"])
    meta["theme"] = "\n".join(meta["theme"])
    return meta


def init_legal_practices_db():
    """
    Initializes ChromaDB vector storage with legal practices docs.
    """
    persistent_client = chromadb.PersistentClient(path=str(CHROMADB_STORAGE_FOLDER))
    collection = persistent_client.create_collection(
        name="legal_practices", get_or_create=True, metadata={"hnsw:space": "cosine"}
    )

    # Load doc uids
    logging.info("Start loading doc uids.")
    with open(LEGAL_PRACTICES_DATA_FOLDER / "summaries_emb_uids.pkl", "rb") as f:
        doc_uids = pickle.load(f)
    logging.info("Successfully load doc uids.")

    # Load embeddings
    logging.info("Start loading embeddings.")
    embs = np.load(LEGAL_PRACTICES_DATA_FOLDER / "summaries_emb.npy")
    logging.info("Successfully load embeddings.")

    # Load metadatas (dict with docs uid and metadata (codex and theme))
    logging.info("Start loading metadatas.")
    with open(LEGAL_PRACTICES_DATA_FOLDER / "doc2meta.json", "r") as f:
        metadatas = json.load(f)
    logging.info("Successfully load metadatas.")

    # Load summaries (actual docs)
    logging.info("Start loading summaries.")
    summaries_path = LEGAL_PRACTICES_DATA_FOLDER / "summaries"
    summaries = {}
    for summary_path in tqdm(summaries_path.iterdir()):
        with open(summary_path, "r") as f:
            summaries[summary_path.stem] = f.read()
    logging.info("Successfully load summaries.")

    # Add data to the collection
    collection.add(
        ids=doc_uids,
        embeddings=embs,
        metadatas=[squeeze_meta(metadatas[uid]) for uid in doc_uids],
        documents=[summaries[uid] for uid in doc_uids],
    )

    return persistent_client, collection

In [4]:
%%time
chroma_client, legal_practices_collection = init_legal_practices_db()

INFO:root:Start loading doc uids.
INFO:root:Successfully load doc uids.
INFO:root:Start loading embeddings.
INFO:root:Successfully load embeddings.
INFO:root:Start loading metadatas.
INFO:root:Successfully load metadatas.
INFO:root:Start loading summaries.
32830it [00:18, 1800.36it/s]
INFO:root:Successfully load summaries.


CPU times: user 1min 48s, sys: 21.2 s, total: 2min 9s
Wall time: 2min 29s


## Testing

In [5]:
chroma_client.list_collections()

[Collection(name=legal_practices), Collection(name=legal_docs)]

In [6]:
legal_practices_collection.count()

18052

In [7]:
legal_practices_collection.get("gi9gPiMIqsQV")

{'ids': ['gi9gPiMIqsQV'],
 'embeddings': None,
 'documents': ['### Резюме судебного решения\n\n#### 1. Основные сведения о деле:\n\n- **Регулируется:** Гражданским кодексом РФ (ГК РФ), Налоговым кодексом РФ (НК РФ), Конституцией РФ.\n- **Тематика:** Спор о законности предпринимательской деятельности.\n\n#### 2. Стороны и обстоятельства дела:\n\n- **Истец:** Прокурор Акушинского района Республики Дагестан.\n- **Ответчик:** Магомедов Шамиль Гаджиевич.\n- **Ключевые обстоятельства:** Магомедов осуществлял торговую деятельность без государственной регистрации в качестве индивидуального предпринимателя, что является нарушением федерального законодательства.\n\n#### 3. Позиции сторон:\n\n- **Истец:** Прокурор утверждал, что Магомедов не зарегистрирован в качестве индивидуального предпринимателя, но занимается торговой деятельностью, что является нарушением законодательства. Просил признать деятельность незаконной и прекратить ее.\n- **Ответчик:** Магомедов признал иск и подтвердил, что дейст